In [1]:
import general
import tree.halomodule as hmo

In [28]:
def extract_halos_within(halos, i_center, info, dist_in_mpc=1.0):
    xc = halos['x'][i_center]
    yc = halos['y'][i_center]
    zc = halos['z'][i_center]

    xx = halos['x']
    yy = halos['y']
    zz = halos['z']

    dd = np.multiply(distance_to([xc,yc,zc], [xx,yy,zz]), info.pboxsize)

    return (dd < (dist_in_mpc))


def distance_to(xc, xx):
    import numpy as np
    return np.sqrt([(xc[0] - xx[0])**2 + (xc[1] - xx[1])**2 + (xc[2] - xx[2])**2])[0]


def all_gals_org(treedata, final_gals, nout_ini=None, nout_fi=None):
    """
       build a list of all progenitors of the final_gals from nout_fi up to nout_ini
       [ [final_gals (at nout = nout_fi) ], 
         [all progenitors of final gals at nout = nout_fi -1],
         [ '' at the at nout = nout_fi -2 ], ...]
    """
    if nout_ini == None:
        nout_ini = min(treedata['nout'])
    if nout_fi == None:
        nout_fi = max(treedata['nout'])

    all_gals_at_nouts = []
    for inout, nout in enumerate(range(nout_ini, nout_fi+1)):
        all_gals_this_nout = []
        tree_now = treedata[np.where(treedata['nout'] == nout)]

        for finalgal in final_gals:
            i_gals_include = np.where(tree_now['tree_root_id'] == finalgal)[0]
            [all_gals_this_nout.append(gal) for gal in tree_now['id'][i_gals_include]]

        all_gals_at_nouts.append(all_gals_this_nout)

    return all_gals_at_nouts


def all_gals(treedata, final_gals, nout_ini=None, nout_fi=None):
    """
       build a list of all progenitors of the final_gals from nout_fi up to nout_ini
       [ [final_gals (at nout = nout_fi) ], 
         [all progenitors of final gals at nout = nout_fi -1],
         [ '' at the at nout = nout_fi -2 ], ...]
    """
    if nout_ini == None:
        nout_ini = min(treedata['nout'])
    if nout_fi == None:
        nout_fi = max(treedata['nout'])

    #all_gals_at_nouts = []
    inds=[]
    for inout, nout in enumerate(range(nout_ini, nout_fi+1)):
        all_gals_this_nout = []
        tree_now = treedata[np.where(treedata['nout'] == nout)]

        for finalgal in final_gals:
            inds.extend(np.where(treedata['tree_root_id'] == finalgal)[0])
            #i_gals_include = np.where(tree_now['tree_root_id'] == finalgal)[0]
            #[all_gals_this_nout.append(gal) for gal in tree_now['id'][i_gals_include]]

        #all_gals_at_nouts.append(all_gals_this_nout)

    return treedata[inds]


def halo_from_tree(tree_element, info):
    import tree.halomodule as hmo

    dtype_halo = [('id', '<i4'), ('idx', '<i4'), ('m', '<f4'), ('mvir', '<f4'),
              ('r', '<f4'), ('rvir', '<f4'),
              ('x', '<f4'), ('y', '<f4'), ('z', '<f4'),
              ('vx', '<f4'), ('vy', '<f4'), ('vz', '<f4')]

    cboxsize = 200.

    nout = tree_element['nout']
    h = hmo.Halo(nout=nout, halofinder='HM', info=info, is_gal=True)

    h.data = np.recarray(len(tree_element), dtype=dtype_halo)
    h.nout = nout

    h.data['m'] = tree_element['m']
    h.data['mvir'] = tree_element['mvir']
    h.data['x'] = tree_element['x'] / cboxsize
    h.data['y'] = tree_element['y'] / cboxsize
    h.data['z'] = tree_element['z'] / cboxsize # Mpc/h -> code unit
    h.data['vx'] = tree_element['vx']
    h.data['vy'] = tree_element['vy']
    h.data['vz'] = tree_element['vz']
    h.data['r'] = tree_element['r'] # already in code unit
    h.data['rvir'] = tree_element['rvir'] / (cboxsize * 1000) # kpc/h -> code unit
    h.data['id'] = tree_element['Orig_halo_id']
    h.data['idx'] = tree_element['id']
    h.aexp = tree_element['aexp']

    return h

def dist(data, center):
    return np.sqrt(np.square(center['x'] - data['x']) +
                np.square(center['y'] - data['y']) +
                np.square(center['z'] - data['z']))

def dist2(data, center):
    return (np.square(center['x'] - data['x']) +
         np.square(center['y'] - data['y']) +
         np.square(center['z'] - data['z']))


def find_halo_near(data, center, rscale=1.0):
    import numpy as np
    i_gal_ok = dist2(data, center) <  np.square(rscale * center['rvir'])
    return i_gal_ok


def unique(a,b):
    a = np.concatenate((a,b))
    a = np.sort(a)
    b = np.diff(a)
    b = np.r_[1, b]
    return a[b != 0]



In [29]:
gd = general.defaults.Default()

In [30]:
import numpy as np
import load
import pickle
import tree.ctutils as ctu
wdir='./10002/'

nout_ini=57
nout_fi = 187
is_gal = True
r_cluster_scale = 2.9
m_halo_min = 1e10
nout_complete = 87


info = load.info.Info(nout=nout_fi, base=wdir, load=True)
# load galaxy tree 

alltrees = pickle.load(open(wdir + \
                gd.dir_galaxy_tree + "extended_tree.pickle", "rb"))

td = alltrees.data

# halo catalog
hhal = hmo.Halo(base=wdir, nout=nout_fi, halofinder='HM', load=True, is_gal=False)
# cluster radius 

i_center = np.where(hhal.data['np'] == max(hhal.data['np']))[0]
r_cluster = hhal.data['rvir'][i_center].squeeze() * hhal.info.pboxsize

# galaxies
hh = hmo.Halo(base=wdir, nout=nout_fi, halofinder='HM', info=info, load=True, is_gal=is_gal)
i_center = np.where(hh.data['np'] == max(hh.data['np']))[0]
# galaxies that are within r_cluster_scale * the cluster halo from the BCG 
# (not the cluster halo center, although two must be virtually identical)

# All galaxies inside the cluster radius * r_scale
i_satellites = extract_halos_within(hh.data, i_center, info, dist_in_mpc = r_cluster * r_cluster_scale)
print("Total {} galaxies \n {} galaxies are within {} times the cluster virial radius, {} Mpc".format(
      len(i_satellites),sum(i_satellites), r_cluster_scale, r_cluster_scale * r_cluster))

# Above a mass cut at nout_fi
# halos found inside the cluster and have complete tree back to nout_ini
large_enough = hh.data['mvir'] > m_halo_min
halo_list = hh.data['id'][i_satellites * large_enough]
final_ids = ctu.check_tree_complete(td, nout_complete, nout_fi, halo_list, idx=False) # 87: z = 1


# build list of progenitors (all mass)
tt_final = td[td['nout'] == nout_fi]
final_gals_idx = [tt_final['id'][tt_final['Orig_halo_id'] == final_gal] for final_gal in final_ids]
ngals = len(final_gals_idx)
print(" {} galaxies have complete tree up to nout = {}".format(ngals, nout_complete))
# Search for all galaxies that are listed in the trees of final_gals
prg_only_tree = all_gals(td, final_gals_idx, nout_fi=nout_fi, nout_ini=nout_ini)
#all_gals_in_trees = prg_only_tree['']

Total 474 galaxies 
 337 galaxies are within 2.9 times the cluster virial radius, 4.199010105870456 Mpc
 58 galaxies have complete tree up to nout = 87


/home/hoseung/.local/lib/python3.5/site-packages/ipykernel/__main__.py:65: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [34]:
prg_only_tree['Orig_halo_id']

array([  1, 519,   1, ..., 877, 811, 883])

In [40]:
prg_only_tree['Orig_halo_id']

import utils.match as mtc
nout=186

mstar_min = 5e9

inout=1

for inout, nout in enumerate(range(nout_fi)
# idx list of galaxies at the current nout (from all_gals_in_trees list)
t_now = prg_only_tree[td['nout'] == nout]
#idxs_tree_now = all_gals_in_trees[inout]

#gals_in_tree_now = halo_from_tree(t_now[mtc.match_list_ind(t_now['id'], np.array(idxs_tree_now))], info)
gals_in_tree_now = prg_only_tree[prg_only_tree['nout'] == nout]
id_now = gals_in_tree_now['Orig_halo_id'] # this is Orig_halo_id

allhal = hmo.Halo(base=wdir, nout=nout, is_gal=False, halofinder='HM', return_id=False, load=True)
cluster_now = allhal.data[allhal.data.np.argmax()]

dd = np.sqrt(np.square(cluster_now['x'] * 200 - t_now['x']) + 
             np.square(cluster_now['y'] * 200 - t_now['y']) + 
             np.square(cluster_now['z'] * 200 - t_now['z'])) # in Mpc/h
print(max(dd))




# Galaxy with enough stellar mass
# -> there is no stellar mass cut when building all_gals_in_trees list.
allgal = hmo.Halo(base=wdir, nout=nout, is_gal=True, halofinder='HM', return_id=False, load=True)
igal_mass_ok_cat = allgal.data['m'] > mstar_min
print("(catalogue) # galaxies more massive than {:.2e} at nout = {}".format(mstar_min, sum(igal_mass_ok_cat)))

# Galaxies outside the refinement region MUST be excluded.
# But, do you know the refinement region? 
# any galaxy closer than the furthest galaxy in the tree is OK.
#igals_in_tree_now = mtc.match_list_ind(allgal.data['id'], id_now)
dist_from_cluster_center = dist(allgal.data, cluster_now)

(catalogue) # galaxies more massive than 5.00e+09 at nout = 144


/home/hoseung/.local/lib/python3.5/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 4268766 but corresponding boolean dimension is 83318


In [52]:
cluster_now['z'] * info.pboxsize

157.13034425035198

In [54]:


#max_dist_prgs = 

13.2849156683


In [39]:
igal_add = (allgal.data['m'] > mstar_min) * (dist_from_cluster_center < max(dist_from_cluster_center[igals_in_tree_now]))
# massvie and inside zoomin + tree pogenitors
igal_ok = unique(igal_add,igals_in_tree_now)

(catalogue) # galaxies more massive than 5.00e+09 at nout = 144


/home/hoseung/.local/lib/python3.5/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 4268766 but corresponding boolean dimension is 83318


ValueError: max() arg is an empty sequence

In [38]:
gals_in_tree_now.dtype

dtype([('aexp', '<f8'), ('id', '<i8'), ('desc_aexp', '<i8'), ('desc_id', '<i8'), ('nprog', '<i8'), ('pid', '<i8'), ('upid', '<i8'), ('desc_pid', '<i8'), ('phantom', '<i8'), ('sam_mvir', '<f8'), ('mvir', '<f8'), ('rvir', '<f8'), ('rs', '<i8'), ('vrms', '<f8'), ('mmp', '<f8'), ('aexp_last_MM', '<f8'), ('vmax', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<i8'), ('jx', '<i8'), ('jy', '<i8'), ('jz', '<f8'), ('spin', '<i8'), ('b_id', '<i8'), ('d_id', '<i8'), ('tree_root_id', '<i8'), ('Orig_halo_id', '<i8'), ('nout', '<i8'), ('next_coprogenitor_d_id', '<i8'), ('last_progenitor_d_id', '<f8'), ('np', '<i4'), ('m', '<f4'), ('r', '<f4'), ('tvir', '<f4'), ('cvel', '<f4'), ('sig', '<f4'), ('sigbulge', '<f4'), ('mbulge', '<f4')])

In [ ]:
# at each nout, add galaxies above certain mass cut(least mass main progenitor)
# and closer to the BCG than the most distant progenitor(, which must be inside the zoom-in region)
mstar_min = 5e9
for nout in range(180, nout_fi):
    allgal = hmo.Halo(base=wdir, nout=nout, is_gal=True, halofinder='HM', return_id=False, load=True)
    igal_mass_ok = allgal.data['m'] > mstar_min
    print("# galaxies more massive than {:.2e} at nout = {}".format(mstar_min, sum(igal_mass_ok)))

    # Galaxies outside the refinement region MUST be excluded.
    # But, do you know the refinement region? 
    # any galaxy closer than the furthest galaxy in the tree is OK.
    igals_in_tree_now = mtc.match_list_ind(allgal.data['id'], id_now)
    dist_from_cluster_center = dist(allgal.data, cluster_now)
    
    
    #np.unique()


# Then, I don't need to worry about sampling anymore. 

# along with the list, save anoter list of "tree_root_halo" to make it easy to identify the final descendent.